In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from crud import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "SNHU1234"

# Additional Variables that would be used for the connection to the MongoDB
HOST = 'nv-desktop-services.apporto.com'
PORT = 32299
DB = 'AAC'
COL = 'animals'

# Connect to database via CRUD Module
db = AnimalShelter(username, password, HOST, PORT, DB, COL)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({'animal_type': 'Dog'}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'GSL.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Div(className='row',
            style={'display' : 'flex', 'margin' : '10px'},
            children = [
                html.Img(src = 'data:image/png;base64,{}'.format(encoded_image.decode()), style={'height':'5%', 'width':'5%'}),
                html.Div(
                    children=[
                        html.Center(html.B(html.H1('CS-340 Dashboard', style = {'textAlign': 'center'}))),
                        html.Center(html.B(html.H2('Created by Nicholas Wyrwas', style={'textAlign': 'center'})))
                    ])
                
            ]
    ),
    html.Hr(),
    html.Div(
        children=[
            dcc.RadioItems(
                id='filter_type',
                options=[
                    {'label': '   Water Rescue   ', 'value': 'Water'},
                    {'label': '   Mountain and Wilderness Rescue   ', 'value': 'Mountain'},
                    {'label': '   Disaster and Individual Rescue   ', 'value': 'Disaster'},
                    {'label': '   Reset   ', 'value': 'Reset'}
                ]
            )
        ]
    ),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
                         editable = False,
                         filter_action = "native",
                         sort_action = "native",
                         sort_mode = "multi",
                         column_selectable = "single",
                         row_selectable = "single",
                         row_deletable = False,
                         selected_columns = [],
                         selected_rows = [0],
                         page_action = "native",
                         page_current = 0,
                         page_size = 10,

                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter_type', 'value')]  # Make sure 'filter_type' matches the ID of the input component
)
    
def update_dashboard(filter_type):
    ## FIX ME Add code to filter interactive data table with MongoDB queries
    if filter_type is None:
        return []

    df = None
    
    if filter_type == 'Water':
        df = pd.DataFrame.from_records(db.read(
            { "$and": [
                {"$or": [
                    {'breed': {"$regex": 'Labrador Retriever Mix'}},
                    {'breed': {"$regex": 'Chesapeake'}},
                    {'breed': {"$regex": 'NewFoundland'}}]},
                {'sex_upon_outcome':'Intact Female'},
                { "$and": [
                    {'age_upon_outcome_in_weeks': {"$gt": 26}},
                    {'age_upon_outcome_in_weeks': {"$gt": 156}}
                ]}]}
        ))
    elif filter_type == 'Mountain':
        df = pd.DataFrame.from_records(db.read(
            { "$and": [
                {"$or": [
                    {'breed': {"$regex": 'German Shepherd'}},
                    {'breed': {"$regex": 'Alaskan Malamute'}},
                    {'breed': {"$regex": 'Old English Sheepdog'}},
                    {'breed': {"$regex": 'Siberian Husky'}},
                    {'breed': {"$regex": 'Rottweiler'}}
                ]},
                {'sex_upon_outcome':'Intact Male'},
                { "$and": [
                    {'age_upon_outcome_in_weeks': {"$gt": 26}},
                    {'age_upon_outcome_in_weeks': {"$gt": 156}}
                ]}]}
        ))
    elif filter_type == 'Disaster':
        df = pd.DataFrame.from_records(db.read(
            { "$and": [
                {"$or": [
                    {'breed': {"$regex": 'Doberman Pinscher'}},
                    {'breed': {"$regex": 'German Shepherd'}},
                    {'breed': {"$regex": 'Golden Retriever'}},
                    {'breed': {"$regex": 'Bloodhound'}},
                    {'breed': {"$regex": 'Rottweiler'}}
                ]},
                {'sex_upon_outcome':'Intact Male'},
                { "$and": [
                    {'age_upon_outcome_in_weeks': {"$gt": 20}},
                    {'age_upon_outcome_in_weeks': {"$gt": 300}}
                ]}]}
        ))
    else:
        filter_type == 'Reset'
        df = pd.DataFrame.from_records(db.read({'animal_type': 'Dog'}))
    
    df.drop(columns=['_id'], inplace=True)
    return df.to_dict('records')

# Display the breeds of animal based on quantity represented in
# the data table
    
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    if viewData is None or len(viewData) == 0:
        return html.P("No data available.")
    df_filtered = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(
            figure=px.pie(df_filtered, names='breed', title='Preferred Animals')
        )
    ]


# This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    # Debug: Print input values
    # print("viewData:", viewData)
    # print("selected index:", index)
    
    if viewData is None or len(viewData) == 0:
        # No data available: Return a default map centered on Austin, TX
        default_center = [30.75, -97.48]
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'},
                   center=default_center, zoom=10, children=[
                       dl.TileLayer(id="base-layer-id")
                   ])
        ]
    
    dff = pd.DataFrame.from_dict(viewData)
    
    # Ensure at least one row is available
    if dff.empty:
        default_center = [30.75, -97.48]
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'},
                   center=default_center, zoom=10, children=[
                       dl.TileLayer(id="base-layer-id")
                   ])
        ]
    
    # Ensure a valid selected row index
    if index is None or len(index) == 0 or index[0] >= len(dff):
        row = 0
    else:
        row = index[0]
    
    default_center = [30.75, -97.48]
    
    # Check if we have enough columns for coordinates (expecting index 13 and 14)
    if len(dff.columns) > 14:
        try:
            lat = dff.iloc[row, 13]
            lon = dff.iloc[row, 14]
            # Validate if coordinates are numbers
            if lat is None or lon is None:
                center = default_center
            else:
                center = [float(lat), float(lon)]
        except Exception as e:
            # If error occurs while extracting coordinates, fallback
            # print("Coordinate extraction error:", e)
            center = default_center
    else:
        center = default_center

    # Ensure tooltip and popup values are available safely:
    tooltip_text = dff.iloc[row, 4] if len(dff.columns) > 4 else "No Breed"
    animal_name = dff.iloc[row, 9] if len(dff.columns) > 9 else "No Name"

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               center=center, zoom=10, children=[
                   dl.TileLayer(id="base-layer-id"),
                   dl.Marker(position=center,
                             children=[
                                 dl.Tooltip(tooltip_text),
                                 dl.Popup([
                                     html.H1("Animal Name"),
                                     html.P(animal_name)
                                 ])
                             ])
               ])
    ]

app.run_server(debug=True)


Dash app running on http://127.0.0.1:13449/
